# Chapter 3: Detect Obstacles and Security

The most  important element in autonomous vehicles is **security**.<br>
To have a safer car, you need, at minimum, two elements:<br>
<ul>
<li>**Obstacle Detection**</li>
<li>**System Failure Measures**</li>
</ul>

## Control the car movement Data Flow

For security reasons, the car shouldn't be moved directly through a typical **/cmd_vel** topic, as you have been doing up until now.

It needs two barriers:
<ol>
<li>**DeadMansSwitch**: When control communication is lost, autonomous vehicles have to move to a safe state. In this simple example, the car must stop in case the movement commands aren't published at a rate higher than 5Hz.</li>
<li>**ObstacleDetection**: When an obstacle is detected, the vehicle must stop until the obstacle is removed. In this simple example, obstacles are detected.</li>
</ol>

This is a diagram of a possible data flow:

![Gazebo Simulation Sensor Image](img/new_catvehicle_graph.png)

This flow can be divided into three parts:<br>

* **External Control Input**: This would be the **/teleoptwist_keyboard** that publishes into the **/cmd_vel** topic. This could be a ROS compatible joystick, keyboard, or something similar. Then, the **/cmdvel_test** node will just redirect the **/cmd_vel** topic to the vehicle's internal **/catvehicle/cmd_vel** topic.<br>
This would be the part implemented inside any action or service that is in charge of moving the vehicle.


* **ObstacleDetection Flow**: The main piece is the **/obstacleStoppercatvehicle** node, which reads the input from the **/catvehicle/cmd_vel** topic and redirects it to the **/catvehicle/cmd_vel_safe** topic, based on the **/distanceEstimator/dist** values. If the distance to the nearest object is less than the minimum accepted, it replaces the **/catvehicle/cmd_vel** values with null values. Otherwise, it redirects **/catvehicle/cmd_vel** without changes to **/catvehicle/cmd_vel_safe**. Here is where the **ObstacleDetection** is implemented.<br>


* **Speed commands to Controller commands**: This part converts the **/catvehicle/cmd_vel_safe** values into Controller Commands that indicate the torque for the rear wheels and the steering actuators. It's here that the **DeadMansSwitch** is implemented. If the **/catvehicle/cmd_vel_safe** is not published at a rate higher than 5Hz, it will replace the values with null values, until the next command is published. 

### System Failure Measures and DeadMansSwitch

In a car there are hundreds of systems dedicated to this topic. But in this course, you will learn the most basic one of them all: **The DeadMansSwitch**.

This system will essentially stop the car if the publication rate is lower than, in this case, 5Hz.
You are going to learn how to implement such a system.

### Obstacle Detection

**GPS** is only useful for localization.<br>
For fast-appearing obstacles, **LaserData** is the best way to go.
For more complex recognition, **Image** is the system used.<br>
With all these sensors, obstacle detection in its many different forms is performed. In this unit, you will work with the most basic of them all: **LaserData**.

<p style="background:#EE9023;color:white;">Exercise U3.1</p>

Create a Python script that contains a publisher that sends commands to the joints of the car and wheels. You can call it **<i>cmdvel2gazebo.py</i>** for future reference.

But it has to **ONLY** send them if it receives the Twist commands at a certain rate.<br>
You will have to implement a **DeadMansSwitch**. You can have different security policies: maybe you can stop the car only until it receives a new message. Or maybe you want a more rigid system that blocks the car for 10 seconds and then tries again. Or it can even block and not work again until an enabling topic is written.

Data you need:<br>
<ul>
<li>The topic where the Twist commands come from is **/catvehicle/cmd_vel_safe**</li>
<li>
The Joint topics to move the car are:<br>
**/catvehicle/front_left_steering_position_controller/command**<br>
**/catvehicle/front_right_steering_position_controller/command**<br>
**/catvehicle/joint1_velocity_controller/command**<br>
**/catvehicle/joint2_velocity_controller/command**<br>
</li>
<li>The angle and speed of the wheels is set by you, based on the Twist messages received. But it would be great if you apply what is known as the **Ackerman Steering**.
If you are unsure of how it works, we highly recommend that you have a look at this video, courtesy of **Jonathan Sprinkle**: https://www.youtube.com/watch?v=i6uBwudwA5o
</li>
</ul>

<p style="background:#EE9023;color:white;">END Exercise U3.1</p>

<p style="background:green;color:white;">Solution Exercise U3.1</p>

Please try to do it by yourself, unless you get stuck or need some inspiration. You will learn much more if you fight for each exercise.

<img src="img/robotignite_logo_text.png"/>

This is an example of how it could be done, courtesy of Jonathan Sprinkle from Arizona Board of Regents:

In [1]:
#!/usr/bin/env python
# 
# Author: Jonathan Sprinkle
# Copyright (c) 2015-2016 Arizona Board of Regents
# All rights reserved.
# 
# Permission is hereby granted, without written agreement and without 
# license or royalty fees, to use, copy, modify, and distribute this
# software and its documentation for any purpose, provided that the 
# above copyright notice and the following two paragraphs appear in 
# all copies of this software.
# 
# IN NO EVENT SHALL THE ARIZONA BOARD OF REGENTS BE LIABLE TO ANY PARTY 
# FOR DIRECT, INDIRECT, SPECIAL, INCIDENTAL, OR CONSEQUENTIAL DAMAGES 
# ARISING OUT OF THE USE OF THIS SOFTWARE AND ITS DOCUMENTATION, EVEN 
# IF THE ARIZONA BOARD OF REGENTS HAS BEEN ADVISED OF THE POSSIBILITY OF 
# SUCH DAMAGE.
# 
# THE ARIZONA BOARD OF REGENTS SPECIFICALLY DISCLAIMS ANY WARRANTIES, 
# INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY 
# AND FITNESS FOR A PARTICULAR PURPOSE. THE SOFTWARE PROVIDED HEREUNDER
# IS ON AN "AS IS" BASIS, AND THE ARIZONA BOARD OF REGENTS HAS NO OBLIGATION
# TO PROVIDE MAINTENANCE, SUPPORT, UPDATES, ENHANCEMENTS, OR MODIFICATIONS.

# This node converts cmd_vel inputs to the vehicle to the ROS topics that
# are exposed in Gazebo for moving the vehicle in simulation. Notably, the
# inputs to Gazebo are to joints on the wheel, so there is a multiplier of
# 2.8101 that is applied to the joint's velocity whenever we try to move
# so that the output in Gazebo will match the desired input velocity.

import rospy
from std_msgs.msg import String, Float64
from geometry_msgs.msg import Twist, Pose
import sys, getopt, math

class cmdvel2gazebo:

    def __init__(self,ns):
        self.ns = ns
        rospy.init_node('cmdvel2gazebo', anonymous=True)

        # the format(ns) looks for the namespace in the ros parameter server, I guess
        
        rospy.Subscriber('/catvehicle/cmd_vel_safe', Twist, self.callback)
        self.pub_steerL = rospy.Publisher('/catvehicle/front_left_steering_position_controller/command', Float64, queue_size=1)
        self.pub_steerR = rospy.Publisher('/catvehicle/front_right_steering_position_controller/command', Float64, queue_size=1)
        self.pub_rearL = rospy.Publisher('/catvehicle/joint1_velocity_controller/command', Float64, queue_size=1)
        self.pub_rearR = rospy.Publisher('/catvehicle/joint2_velocity_controller/command', Float64, queue_size=1)

        # initial velocity and tire angle are 0
        self.x = 0
        self.z = 0

        # TODO: get wheelbase and treadwidth from SDF or 
        #       params database instead of hardcoded here

        # car Wheelbase (in m)
        # simulator value matches the 'real' car
        self.L = 2.62

        # car Tread
        # this value is from the car's manual
        # self.T=1.55

        # car Tread
        # this value is from the simulator
        self.T=1.301

        # how many seconds delay for the dead man's switch
        # TODO: set timeout from launch file or rosparam
        self.timeout=rospy.Duration.from_sec(0.2);
        self.lastMsg=rospy.Time.now()

        # we want maxsteer to be that of the "inside" tire, and since it is 0.6 in gazebo, we
        # set our ideal steering angle max to be less than that, based on geometry
        self.maxsteerInside=0.6;
        # tan(maxsteerInside) = wheelbase/radius --> solve for max radius at this angle
        rMax = self.L/math.tan(self.maxsteerInside);
        # radius of inside tire is rMax, so radius of the ideal middle tire (rIdeal) is rMax+treadwidth/2
        rIdeal = rMax+(self.T/2.0)
        # tan(angle) = wheelbase/radius
        self.maxsteer=math.atan2(self.L,rIdeal)
        # the ideal max steering angle we can command is now set
        rospy.logwarn("######### MAXIMUM ideal steering angle set to =="+str(self.maxsteer))
        

    def callback(self,data):
        # 2.8101 is the gain factor in order to account for mechanical reduction of the tyres
        self.x = 2.8101*data.linear.x
        # constrain the ideal steering angle such that the ackermann steering is maxed out
        self.z = max(-self.maxsteer,min(self.maxsteer,data.angular.z))
        self.lastMsg = rospy.Time.now()

    def publish(self):
        # now that these values are published, we
        # reset the velocity, so that if we don't hear new
        # ones for the next timestep that we time out; note
        # that the tire angle will not change
        # NOTE: we only set self.x to be 0 after 200ms of timeout
        delta_last_msg_time = rospy.Time.now() - self.lastMsg
        msgs_too_old = delta_last_msg_time > self.timeout
        if msgs_too_old:
            #rospy.loginfo(rospy.get_caller_id() + " timed out waiting for new input in /cmd_vel, setting velocity to 0.")
            self.x = 0
            msgRear = Float64()
            msgRear.data = self.x;
            self.pub_rearL.publish(msgRear)
            self.pub_rearR.publish(msgRear)
            return
        #rospy.loginfo("X = "+ str(self.x)+ ", Z = "+ str(self.z))
        # The self.z is the delta angle in radians of the imaginary front wheel of ackerman model.
        if self.z != 0:
            T=self.T
            L=self.L
            # self.v is the linear *velocity*
            r = L/math.fabs(math.tan(self.z))

            rL = r-(math.copysign(1,self.z)*(T/2.0));
            rR = r+(math.copysign(1,self.z)*(T/2.0))
            msgRearR = Float64()
            # the right tire will go a little faster when we turn left (positive angle)
            # amount is proportional to the radius of the outside/ideal
            msgRearR.data = self.x*rR/r;
            msgRearL = Float64()
            # the left tire will go a little slower when we turn left (positive angle)
            # amount is proportional to the radius of the inside/ideal
            msgRearL.data = self.x*rL/r;

            self.pub_rearL.publish(msgRearL)
            self.pub_rearR.publish(msgRearR)

            msgSteerL = Float64()
            msgSteerR = Float64()
            # the left tire's angle is solved directly from geometry
            msgSteerL.data = math.atan2(L,rL)*math.copysign(1,self.z)
            self.pub_steerL.publish(msgSteerL)
    
            # the right tire's angle is solved directly from geometry
            msgSteerR.data = math.atan2(L,rR)*math.copysign(1,self.z)
            self.pub_steerR.publish(msgSteerR)
        else:
            # if we aren't turning, everything is easy!
            msgRear = Float64()
            msgRear.data = self.x;
            self.pub_rearL.publish(msgRear)
            self.pub_rearR.publish(msgRear)

            msgSteer = Float64()
            msgSteer.data = self.z

            self.pub_steerL.publish(msgSteer)
            self.pub_steerR.publish(msgSteer)
def usage():
    print('cmdvel2gazebo -n catvehicle')


def main(argv):
    # we eventually get the ns (namespace) from the ROS parameter server for this node
    ns=''
    node = cmdvel2gazebo(ns)
    rate = rospy.Rate(10) # run at 10Hz
    while not rospy.is_shutdown():
        node.publish()
        rate.sleep()

if __name__ == '__main__':
    main(sys.argv[1:])
    try:
        listener('catvehicle')
    except rospy.ROSInterruptException:
        pass




ModuleNotFoundError: No module named 'rospy'

Here are some interesting elements to be noted:<br>
<ol>
<li>Note that the wheels don't turn symmetrically. See the **"if self.z != 0:"** condition operations. Here, a different angle is given for the inner tire than to the outer one. This is to keep the wheels of the car from exerting unnecessary effort, and to make better turns.
That's, in a nutshell, the **Ackermann Steering model**.<br> 
</li>
<li>
The **DeadMansSwitch** is implemented in this line here:<br>
**<i>if rospy.Time.now() - self.lastMsg > self.timeout:</i>**<br>
This will make the vehicle stop if it doesn't receive updates in the **cmd_vel_safe** commands within the timeout limit. But note that in this model, it just stops until it receives new messages. This will make the car start to stutter when the frequency lowers beneath 5Hz, but it won't stop.
</li>
</ol>

<p style="background:green;color:white;">END Solution Exercise U3.1</p>

<p style="background:#EE9023;color:white;">Exercise U3.2</p>

Create the Obstacle Stopper.<br>
To create it, follow these steps:
* This node will have to read from the **/catvehicle/cmd_vel** topic and publish into the **/catvehicle/cmd_vel_safe** topic. **/catvehicle/cmd_vel_safe** is the topic where your **cmdvel2gazebo.py** script reads the Twist data.
* You will have to read from the **/distanceEstimator/angle** and **/distanceEstimator/dist** topics.
* Based on the readings of the **/distanceEstimator/** topics, this node will have to filter the Twist messages received in the **/catvehicle/cmd_vel** topic.<br>
If the distance is lower than a value, the Twist that you publish in the **/catvehicle/cmd_vel_safe** topic will have to be a STOP message. 
* As EXTRA features, you can try to lower the speed if an obstacle is nearby, or even use the **/distanceEstimator/angle** topic to change the filtering behaviour.

<p style="background:#EE9023;color:white;">END Exercise U3.2</p>

<p style="background:green;color:white;">Solution Exercise U3.2</p>

Please try to do it by yourself, unless you get stuck or need some inspiration. You will learn much more if you fight for each exercise.

<img src="img/robotignite_logo_text.png"/>

This is an example of how it could be done:

In [ ]:
#!/usr/bin/env python
import rospy
from geometry_msgs.msg import Twist
from std_msgs.msg import Float32

class ObstacleStopper(object):
    def __init__(self, minimum_distance=0.0):
        self._minimum_distance = minimum_distance
        rospy.Subscriber("/catvehicle/cmd_vel", Twist, self.cmdvel_callback)
        self.pub_cmdvel_safe = rospy.Publisher('/catvehicle/cmd_vel_safe', Twist, queue_size=0)
        
    def cmdvel_callback(self,data):
        try:
            closest_distance = rospy.wait_for_message('/distanceEstimator/dist', Float32, timeout=1).data
        except:
            rospy.logwarn("Time out /distanceEstimator/dist")
            closest_distance = None
        
        safe_twist = data
        if closest_distance:
            
            object_detected = closest_distance < self._minimum_distance
            if object_detected:
                safe_twist.linear.x = 0.0
            else:
                pass

        else:
            safe_twist.linear.x = 0.0 
        
        self.pub_cmdvel_safe.publish(safe_twist)
        
    
    
    def listener(self):
        rospy.spin()

if __name__ == '__main__':
    rospy.init_node('obstacle_stopper_node', anonymous=True)
    os = ObstacleStopper(minimum_distance= 5.0)
    os.listener()


<p style="background:green;color:white;">END Solution Exercise U3.2</p>

## Congratulations!! 

You now know how to navigate safely without bumping into people and other cars. You also have a security feature that will come in handy if any of the systems break.

The Final Chapter will teach you how all this is done in a real car, using CAN-Bus messages instead of topics to move the car and retrieve the GPS data.